In [1]:
# import sys
# sys.path.append('D:\Documents\pymanopt\examples')
# # sys.path.append('D:\Documents\pymanopt\pymanopt')
# sys.path

In [5]:
%cd D:\Program Files\Python37\Lib\site-packages\pymanopt\examples

D:\Program Files\Python37\Lib\site-packages\pymanopt\examples


In [27]:
%%writefile optimal_rotations.py
# %load optimal_rotations.py

import os

import autograd.numpy as np
# import tensorflow as tf
# import theano.tensor as T
# import torch
from pymanopt.examples._tools import ExampleRunner

import pymanopt
from pymanopt.manifolds import SpecialOrthogonalGroup
from pymanopt.solvers import SteepestDescent


os.environ["TF_CPP_MIN_LOG_LEVEL"] = "2"


SUPPORTED_BACKENDS = (
    "Autograd", "Callable", "PyTorch", "TensorFlow", "Theano"
)


def create_cost_egrad(backend, ABt):
    egrad = None

    if backend == "Autograd":
        @pymanopt.function.Autograd
        def cost(X):
            return -np.tensordot(X, ABt, axes=X.ndim)
    elif backend == "Callable":
        @pymanopt.function.Callable
        def cost(X):
            return -np.tensordot(X, ABt, axes=X.ndim)

        @pymanopt.function.Callable
        def egrad(X):
            return -ABt
    elif backend == "PyTorch":
        ABt_ = torch.from_numpy(ABt)

        @pymanopt.function.PyTorch
        def cost(X):
            return -torch.tensordot(X, ABt_, dims=X.dim())
    elif backend == "TensorFlow":
        X = tf.Variable(tf.zeros(ABt.shape, dtype=np.float64), name="X")

        @pymanopt.function.TensorFlow(X)
        def cost(X):
            return -tf.tensordot(X, ABt, axes=ABt.ndim)
    elif backend == "Theano":
        X = T.tensor3()

        @pymanopt.function.Theano(X)
        def cost(X):
            return -T.tensordot(X, ABt, axes=ABt.ndim)
    else:
        raise ValueError("Unsupported backend '{:s}'".format(backend))

    return cost, egrad


def compute_optimal_solution(ABt):
    n = ABt[0].shape[0]
    U, S, Vt = np.linalg.svd(ABt)
    UVt = U @ Vt
    if abs(1.0 - np.linalg.det(UVt)) < 1e-10:
        return UVt
    # UVt is in O(n) but not SO(n), which is easily corrected.
    J = np.append(np.ones(n - 1), -1)
    return (U * J) @ Vt


def run(backend=SUPPORTED_BACKENDS[0], quiet=False):
    n = 2
    m = 2
    k = 2
    np.random.seed(1)
    A = np.random.randn(k, n, m)
    B = np.random.randn(k, n, m)
    ABt = np.array([Ak @ Bk.T for Ak, Bk in zip(A, B)])

    cost, egrad = create_cost_egrad(backend, ABt)
    manifold = SpecialOrthogonalGroup(n, k)
    problem = pymanopt.Problem(manifold, cost, egrad=egrad)
    if quiet:
        problem.verbosity = 0

    solver = SteepestDescent()
    X = solver.solve(problem)

    if not quiet:
        Xopt = np.array([compute_optimal_solution(ABtk) for ABtk in ABt])
        print('X: ', X)
        print('Xopt: ', Xopt)
        print('A: ', 
              
              A)
        print('B:', B)
        print("Frobenius norm error:", np.linalg.norm(Xopt - X))


if __name__ == "__main__":
    runner = ExampleRunner(run, "Optimal rotations example",
                           SUPPORTED_BACKENDS)
    runner.run()



Overwriting optimal_rotations.py


In [28]:
%run optimal_rotations.py

Optimal rotations example
-------------------------
Using 'Autograd' backend

 iter		   cost val	    grad. norm
    1	+2.0153790075771283e-01	0.00000000e+00
Terminated - min grad norm reached after 1 iterations, 0.04 seconds.

X:  [[[nan]]

 [[nan]]]
Xopt:  [[[1.]]

 [[1.]]]
A:  [[[ 1.62434536]]

 [[-0.61175641]]]
B: [[[-0.52817175]]

 [[-1.07296862]]]
Frobenius norm error: nan


d:\program files\python37\lib\site-packages\pymanopt\solvers\linesearch.py:46: RuntimeWarning: divide by zero encountered in double_scalars
  alpha = self.initial_stepsize / norm_d
d:\program files\python37\lib\site-packages\pymanopt\solvers\linesearch.py:50: RuntimeWarning: invalid value encountered in multiply
  newx = manifold.retr(x, alpha * d)
d:\program files\python37\lib\site-packages\pymanopt\solvers\linesearch.py:55: RuntimeWarning: invalid value encountered in double_scalars
  while (newf > f0 + self.suff_decr * alpha * df0 and
d:\program files\python37\lib\site-packages\pymanopt\solvers\linesearch.py:72: RuntimeWarning: invalid value encountered in double_scalars
  stepsize = alpha * norm_d
